**Lab1 Group 06**</br>
Liang Zhao 260781081</br>
Hongshuo Zhou 260792817

**README**
* The prelude contains the compareResult.c
* There are three parts in this jupyter notebook with
  - 1. Sequential code
  - 2. Parallela1_explicit code
  - 3. Parallela1_unified code
* The running time is displayed at the end of each part
* At the end of each part, there is a bash script containing codes to create executable and check outputs by compareResult.c 




#Prelude
compareResult.c is given by the lab and is used to compare results with the given sol_XXX.txt

In [2]:
%%writefile compareResult.c
#include <stdio.h>
#include <stdlib.h>

void compareFiles(char *file_name1, char *file_name2) 
{ 
//get from https://www.tutorialspoint.com/c-program-to-compare-two-files-and-report-mismatches
FILE* fp1 = fopen(file_name1, "r");
FILE* fp2 = fopen(file_name2, "r");
    // fetching character of two file 
    // in two variable ch1 and ch2 
    char ch1 = getc(fp1); 
    char ch2 = getc(fp2); 
  
    // error keeps track of number of errors 
    // pos keeps track of position of errors 
    // line keeps track of error line 
    int error = 0, pos = 0, line = 1; 
  
    // iterate loop till end of file 
    while (ch1 != EOF && ch2 != EOF) 
    { 
        pos++; 
  
        // if both variable encounters new 
        // line then line variable is incremented 
        // and pos variable is set to 0 
        if (ch1 == '\n' && ch2 == '\n') 
        { 
            line++; 
            pos = 0; 
        } 
  
        // if fetched data is not equal then 
        // error is incremented 
        if (ch1 != ch2) 
        { 
            error++; 
            printf("Line Number : %d \tError"
               " Position : %d \n", line, pos); 
        } 
  
        // fetching character until end of file 
        ch1 = getc(fp1); 
        ch2 = getc(fp2); 
    } 
  
    printf("Total Errors : %d\t", error); 
} 

int main(int argc, char *argv[]){

    if( argc < 3) {
      printf("Require two files\n");
      exit(1);
      
   }
compareFiles(argv[1], argv[2]);
}


Writing compareResult.c


#1. Sequential

##1.1 sequential.cu

In [4]:
%%writefile sequential.cu

#include <string>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define AND 0
#define OR 1
#define NAND 2
#define NOR 3
#define XOR 4
#define NXOR 5

//Evaluates logical expression.
char evaluate_logic_gate(int ops[]) {
    int result=0;
    switch (ops[2]) {
    case NOR:
        result = !(ops[0] | ops[1]);
        break;
    case XOR:
        result = ops[0] ^ ops[1];
        break;
    case NAND:
        result = !(ops[0] & ops[1]);
        break;
    case AND:
        result = ops[0] & ops[1];
        break;
    case OR:
        result = ops[0] | ops[1];
        break;
    case NXOR:
        result = !(ops[0] ^ ops[1]);
        break;
    }
    char output = result + '0';
    return output;
}

//Sequential computation of logical expressions.
//Takes input file path, file length and output file name as inputs.
int main(int argc, char* argv[]) {
    if (argc != 4) {
        printf("Invalid number of inputs, inputs: <input_file_path> <input_file_length> <output_file_length>\n");
        return 1;
    }
    char line[10];

    //store inputs in this variables
    char* input_file_name = argv[1];
    int input_length = atoi(argv[2]);
    char* output_file_name = argv[3];

    FILE* input_file = fopen(input_file_name, "r");
    FILE* output_file = fopen(output_file_name, "w");
    if (input_file == NULL) {
        fprintf(stderr, "Error in reading input file\n");
        return 1;
    }

    //evaluate operands here and start the clock 
    clock_t start = clock();

    int operands[3] = { 0 };
    char result = 0;
    while (fgets(line, 10, input_file)) {
        operands[0] = atoi(line);
        operands[1] = atoi(line + 2);
        operands[2] = atoi(line + 4);
        result = evaluate_logic_gate(operands);
        fprintf(output_file, "%c\n", result);
    }

    clock_t end = clock();
    double time_taken = ((double) end - start) / CLOCKS_PER_SEC;
    printf("Simulate logic gate sequentially for %s takes %f seconds to execute \n", input_file_name, time_taken);

    fclose(output_file);
    fclose(input_file);

    return 0;

}

Overwriting sequential.cu


##1.2 Bash file
Bash file to run sequrntial.cu
</br>Compare results with solution files by using compareResult.c

In [22]:
%%script bash

#run sequential.cu
nvcc -o sequential sequential.cu
./sequential input_1000000.txt 10000 output_1000000.txt

#compare results
gcc -o compareResult compareResult.c
./compareResult sol_1000000.txt output_1000000.txt

Simulate logic gate sequentially for input_1000000.txt takes 0.138774 seconds to execute 
Total Errors : 0	



```
# 此内容为代码格式
```

#2. Parallel1_explicit

##2.1 parallel_explicit.cu

In [25]:
%%writefile parallel_explicit.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <chrono>
#include <device_launch_parameters.h>
#include <string.h>

#define AND 0
#define OR 1
#define NAND 2
#define NOR 3
#define XOR 4
#define NXOR 5
#define THREADS_PER_BLOCK 1024

__global__ void evaluate_logic_gate(char* input_buffer, int input_length, char* output_buffer, int input_line_length, int output_line_length) {
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  if (index < input_length) {
        char* logic_line = input_buffer + input_line_length * (index);
        int op1 = logic_line[0] - '0';
        int op2 = logic_line[2] - '0';
        int gate_type = logic_line[4] - '0';
        char result;

        switch (gate_type) {
        case NOR:
            result = !(op1 | op2);
            break;
        case XOR:
            result = op1 ^ op2;
            break;
        case NAND:
            result = !(op1 & op2);
            break;
        case AND:
            result = op1 & op2;
            break;
        case OR:
            result = op1 | op2;
            break;
        case NXOR:
            result = !(op1 ^ op2);
            break;
        }

        char* output_location = output_buffer + index * output_line_length;
        char output_value = result + '0';

        output_location[0] = output_value;
        output_location[1] = '\0';
        output_location[2] = '\n';
    }
}

//Sequential computation of logical expressions: explicit memory
//Takes input file path, file length and output file name as inputs.
int main(int argc, char* argv[]) {
    if (argc != 4) {
        printf("Invalid number of inputs, inputs: <input_file_path> <input_file_length> <output_file_length>\n");
        return 1;
    }

    int input_line_length = 7;
    int output_line_length = 3;

    char* input_fileName = argv[1];
    int input_length = atoi(argv[2]);
    char* output_fileName = argv[3];

    FILE* inputFile = fopen(input_fileName, "r");
    if (inputFile == NULL) {
        fprintf(stderr, "Error in reading input file\n");
        return 1;
    }

    //allocate host memery and initiate host data
    int inputSize = input_length * input_line_length * sizeof(unsigned char);
    int outputSize = input_length * output_line_length * sizeof(unsigned char);
    char* input_buffer = (char*)malloc(inputSize);

    char buf[7];
    int addressLocation = 0;
    while (fgets(buf, sizeof buf, inputFile) != NULL) {
        strcpy(input_buffer + addressLocation, buf);
        addressLocation += input_line_length;
    }
    fclose(inputFile);

    //allocate device memory
    char* cudaBuffer;
    char* output_buffer;
    char* returnBuffer;
    returnBuffer = (char*)malloc(outputSize * sizeof(char));
    cudaMalloc(&cudaBuffer, inputSize);
    cudaMalloc(&output_buffer, outputSize);

clock_t start = clock();
    //transfer input data from host to device memory
    cudaMemcpy(cudaBuffer, input_buffer, inputSize, cudaMemcpyHostToDevice);
clock_t end = clock();
   // clock_t start = clock();

    //execute in kernel
    evaluate_logic_gate << < (input_length + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK, THREADS_PER_BLOCK >> > (cudaBuffer, input_length, output_buffer, input_line_length, output_line_length);

   // clock_t end = clock();
    double time_taken = ((double) end - start) / CLOCKS_PER_SEC;

    cudaDeviceSynchronize();

    //transfer output from device memory to host memory
    cudaMemcpy(returnBuffer, output_buffer, outputSize, cudaMemcpyDeviceToHost);

    FILE* outputFile = fopen(output_fileName, "w");
    int counter = 0;
    addressLocation = 0;

    while (counter < input_length) {
        if (counter == input_length - 1) {
            fprintf(outputFile, "%s", returnBuffer + addressLocation);
        }
        else {
            fprintf(outputFile, "%s\n", returnBuffer + addressLocation);
        }
        addressLocation += output_line_length;
        counter++;
    }

    fclose(outputFile);

    free(input_buffer);
    free(returnBuffer);
    cudaFree(cudaBuffer);
    cudaFree(output_buffer);

    printf("Simulate logic gate parallel_explicitly for %s takes %f seconds to execute kernel function \n", input_fileName, time_taken);

    return 0;
}

Overwriting parallel_explicit.cu


##2.2 Bash file
Bash file to run parallel_explicit.cu
</br>Compare results with solution files by using compareResult.c

In [30]:
%%script bash

#run parallela1_explicit.cu
nvcc -o parallela1_explicit parallel_explicit.cu
./parallela1_explicit input_10000.txt 10000 output_10000.txt

#compare results
gcc -o compareResult compareResult.c
./compareResult sol_10000.txt output_10000.txt

Simulate logic gate parallel_explicitly for input_10000.txt takes 0.000033 seconds to execute kernel function 
Total Errors : 0	

#3. Parallel1_unified

##3.1 parallel_unified.cu

In [1]:
%%writefile parallel_unified.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <chrono>
#include <device_launch_parameters.h>
#include <string.h>

#define AND 0
#define OR 1
#define NAND 2
#define NOR 3
#define XOR 4
#define NXOR 5
#define THREADS_PER_BLOCK 1024


__global__ void evaluate_logic_gate(char* input_buffer, int input_length, char* output_buffer, int input_line_length, int output_line_length) {
    int index = threadIdx.x + blockIdx.x * blockDim.x;

    if (index < input_length) {
        char* logic_line = input_buffer + input_line_length * (index);
        int op1 = logic_line[0] - '0';
        int op2 = logic_line[2] - '0';
        int gateType = logic_line[4] - '0';
        char result;

        switch (gateType) {
        case NOR:
            result = !(op1 | op2);
            break;
        case XOR:
            result = op1 ^ op2;
            break;
        case NAND:
            result = !(op1 & op2);
            break;
        case AND:
            result = op1 & op2;
            break;
        case OR:
            result = op1 | op2;
            break;
        case NXOR:
            result = !(op1 ^ op2);
            break;
        }

        char* output_location = output_buffer + index * output_line_length;
        char output_value = result + '0';

        output_location[0] = output_value;
        output_location[1] = '\0';
        output_location[2] = '\n';
    }
}

//Sequential computation of logical expressions: Unified Memory
//Takes input file path, file length and output file name as inputs.
int main(int argc, char* argv[]) {

    if (argc != 4) {
        printf("Invalid number of inputs, inputs: <input_file_path> <input_file_length> <output_file_length>\n");
        return 1;
    }

    int input_line_length = 7;
    int output_line_length = 3;
    char* input_file_name = argv[1];
    int input_length = atoi(argv[2]);
    char* output_file_name = argv[3];

    FILE* inputFile = fopen(input_file_name, "r");
    if (inputFile == NULL) {
        fprintf(stderr, "Error in reading input file\n");
        return 1;
    }

    //allocate host memery and initiate host data
    int inputSize = input_length * input_line_length * sizeof(unsigned char);
    int outputSize = input_length * output_line_length * sizeof(unsigned char);
    char* input_buffer = (char*)malloc(inputSize);

    char buf[7];
    int addressLocation = 0;
    while (fgets(buf, sizeof buf, inputFile) != NULL) {
        strcpy(input_buffer + addressLocation, buf);
        addressLocation += input_line_length;
    }

    fclose(inputFile);

    char* cudaBuffer;
    char* output_buffer;

    // Allocate Unified Memory -- accessible from CPU or GPU
    cudaMallocManaged(&cudaBuffer, inputSize);
    cudaMallocManaged(&output_buffer, outputSize);
    for (int i = 0; i < inputSize; i++) {
        cudaBuffer[i] = input_buffer[i];
    }

    clock_t start = clock();

    //execute in kernel
    evaluate_logic_gate<<< (input_length + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK, THREADS_PER_BLOCK >> > (cudaBuffer, input_length, output_buffer, input_line_length, output_line_length);
    cudaDeviceSynchronize();

    clock_t end = clock();
    double timeTaken = ((double) end - start) / CLOCKS_PER_SEC;
   
    FILE* outputFile = fopen(output_file_name, "w");
    int counter = 0;
    addressLocation = 0;
    while (counter < input_length) {
        if (counter == input_length - 1) {
            fprintf(outputFile, "%s", output_buffer + addressLocation);
        }
        else {
            fprintf(outputFile, "%s\n", output_buffer + addressLocation);
        }
        addressLocation += output_line_length;
        counter++;
    }
    fclose(outputFile);
    // Free memory
    free(input_buffer);
    cudaFree(cudaBuffer);
    cudaFree(output_buffer);

    printf("Simulate logic gate parallel_unified for %s takes %f seconds to execute kernel function \n", input_file_name, timeTaken);
    return 0;
}

Writing parallel_unified.cu


##3.2 Bash file
Bash file to run parallel_unified.cu
</br>Compare results with solution files by using compareResult.c

In [4]:
%%script bash

#run parallela1_explicit.cu
nvcc -o parallela1_unified parallel_unified.cu
./parallela1_unified input_1000000.txt 1000000 output_1000000.txt

#compare results
gcc -o compareResult compareResult.c
./compareResult sol_10000.txt output_10000.txt

Simulate logic gate parallel_unified for input_1000000.txt takes 0.002303 seconds to execute kernel function 


bash: line 8:   370 Segmentation fault      (core dumped) ./compareResult sol_10000.txt output_10000.txt
